In [ ]:
# train and evluatw
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)
from mmcv import Config
import mmcv
from mmdet.apis import set_random_seed
# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv.runner import load_checkpoint
from mmdet.apis import multi_gpu_test, single_gpu_test
print(get_compiling_cuda_version())
print(get_compiler_version())
from mmdet.datasets import build_dataset, build_dataloader
from mmdet.models import build_detector
from mmdet.apis import train_detector
import copy
import os.path as osp
from mmcv.parallel import MMDataParallel
import mmcv
import numpy as np
import json
import os
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

In [ ]:
import os.path as osp

def convert_breast_to_coco(ann_file, out_file, image_prefix):
    '''
    This function is for converting breast json file into coco version
    ann_file: is the json file you create in preprocess
    out_file: is the output file name
    image_prefix: is the prefix path 
    '''
    data_infos = mmcv.load(ann_file)

    annotations = []
    images = []
    obj_count = 0
    for idx, v in enumerate(mmcv.track_iter_progress(data_infos.values())):
        filename = v['file_name']
        img_path = osp.join(image_prefix, filename)
        height, width = mmcv.imread(img_path).shape[:2]

        images.append(dict(
            id=idx,
            file_name=filename,
            height=height,
            width=width))
        bboxes = []
        labels = []
        masks = []
        for _,obj in enumerate(v['annotations']):
            data_anno = dict(
                    image_id=idx,
                    id=obj_count,
                    category_id=0,
                    bbox=[obj['bbox'][0],obj['bbox'][1],obj['bbox'][2] -obj['bbox'][0],obj['bbox'][3] -obj['bbox'][1]],
                    area=(obj['bbox'][2] -obj['bbox'][0]) * (obj['bbox'][3] -obj['bbox'][1]),
                    segmentation=[],
                    iscrowd=0)
            annotations.append(data_anno)
            obj_count += 1
    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{'id':0, 'name': 'cancer'}])
    mmcv.dump(coco_format_json, out_file)
convert_balloon_to_coco('path to/breast/images/train/sample.json', 'path to/breast/images/train/annotation_coco.json', 'path to/breast/')
convert_balloon_to_coco('path to/breast/images/val/sample.json', 'path to/breast/images/val/annotation_coco.json', 'path to/breast/')

In [ ]:
# output the dataset description
cfg = Config.fromfile('./configs/breast/faster_rcnn_r50_fpn_ohem_2x_coco_breast.py')
datasets = [build_dataset(cfg.data.train)]
datasets

## Faster RCNN C4 model

In [ ]:
# train the model
cfg = Config.fromfile('./configs/breast/faster_rcnn_r50_caffe_c4_2x_breast.py')
# Set up working dir to save files and logs
cfg.work_dir = './work_dirs/faster_rcnn_r50_caffe_c4_2x_breast'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.0025
#cfg.lr_config.warmup = 'linear'
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
#cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [1]
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('val_cfg'))
model.init_weights()
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
# evaluate the model
cfg = Config.fromfile('./configs/breast/faster_rcnn_r50_caffe_c4_2x_breast.py')
# Set up working dir to save files and logs
cfg.work_dir = './work_dirs/faster_rcnn_r50_caffe_c4_2x_breast'
dataset = build_dataset(cfg.data.test)
checkpoint = cfg.work_dir +'/latest.pth'
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.val.pop('samples_per_gpu', 1),
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
model.CLASSES = checkpoint['meta']['CLASSES']
model = MMDataParallel(model, device_ids=[2])
outputs = single_gpu_test(model, data_loader)
metric = dataset.evaluate(outputs, metric='bbox',classwise=True)

## Faster RCNN RPN

In [ ]:
# train the model
cfg = Config.fromfile('./configs/breast/faster_rcnn_r50_fpn_2x_breast.py')
# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_2x_breast'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.0025
#cfg.lr_config.warmup = 'linear'
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
#cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [1]
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.init_weights()
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
# evaluate the model
cfg = Config.fromfile('./configs/breast/faster_rcnn_r50_fpn_2x_breast.py')
# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_2x_breast'
dataset = build_dataset(cfg.data.test)
checkpoint = cfg.work_dir +'/latest.pth'
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.val.pop('samples_per_gpu', 1),
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
model.CLASSES = checkpoint['meta']['CLASSES']
model = MMDataParallel(model, device_ids=[2])
outputs = single_gpu_test(model, data_loader)
metric = dataset.evaluate(outputs, metric='bbox',classwise=True)

## test on a single slice

In [ ]:
img = mmcv.imread('path to/breast/images/val/xxx.png')
model.cfg = cfg
result = inference_detector(model, img)
show_result_pyplot(model, img, result,score_thr=0.5)